In [2]:
import os
import re
import nltk
import string
import pandas as pd

from pickle import load
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist, classify, NaiveBayesClassifier

In [3]:
input = open(os.path.join('..', 'storage', 'mac_morpho.1.0.0.pkl'), 'rb')
tagger = load(input)
input.close()

In [6]:
df = pd.read_csv(os.path.join('..', 'storage', 'base_sentiment.1.0.2.csv'))
md = pd.read_csv(os.path.join('..', 'storage', 'DADOS_ABERTOS_MEDICAMENTOS.csv'), sep=';', encoding='latin-1')

In [7]:
md

,TIPO_PRODUTO,NOME_PRODUTO,DATA_FINALIZACAO_PROCESSO,CATEGORIA_REGULATORIA,NUMERO_REGISTRO_PRODUTO,DATA_VENCIMENTO_REGISTRO,NUMERO_PROCESSO,CLASSE_TERAPEUTICA,EMPRESA_DETENTORA_REGISTRO,SITUACAO_REGISTRO,PRINCIPIO_ATIVO
0,MEDICAMENTO,(VITAMINAS A ) + ASSSOCIACÕES,25/04/2000,SIMILAR,104540166,01/04/2005,25000025416,VITAMINAS E SUPLEMENTOS MINERAIS,60874187000184 - DAIICHI SANKYO BRASIL FARMACÊ...,CADUCO/CANCELADO,NaN
1,MEDICAMENTO,AC SALICILICO + AC BENZOICO + IODO,19/05/2000,SIMILAR,119350001,01/05/2005,25010171858,ANTIMICOTICOS PARA USO TOPICO,63826069000199 - LABORATORIO FLORA DA AMAZONIA...,CADUCO/CANCELADO,NaN
2,MEDICAMENTO,ALENDRONATO SODICO,04/06/2001,SIMILAR,100470305,01/05/2006,25351022805,SUPRESSORES DA REABSORCAO OSSEA,61286647000116 - SANDOZ DO BRASIL INDÚSTRIA FA...,CADUCO/CANCELADO,NaN
3,MEDICAMENTO,ARNICA MONTANA,19/03/2002,FITOTERÁPICO,104730021,01/03/2007,25025007821,FITOTERAPICO SIMPLES,92762277000170 - VIDORA FARMACÊUTICA LTDA,CADUCO/CANCELADO,NaN
4,MEDICAMENTO,ARNICA MONTANA L.,01/07/2002,FITOTERÁPICO,131750004,01/06/2007,25351032631,FITOTERAPICO SIMPLES,00925697000101 - LIMED LABORATORIO INDUSTRIAL ...,CADUCO/CANCELADO,NaN
...,...,...,...,...,...,...,...,...,...,...,...
30742,MEDICAMENTO,CEFAZOLINA SÓDICA,12/06/2023,GENÉRICO,174200014,01/01/2028,25351277352,CEFALOSPORINAS,44639493000180 - EUGIA PHARMA INDUSTRIA FARMAC...,VÁLIDO,CEFAZOLINA SÓDICA
30743,MEDICAMENTO,CEFTRIAXONA SÓDICA,12/06/2023,GENÉRICO,174200016,01/10/2028,25351278645,CEFALOSPORINAS,44639493000180 - EUGIA PHARMA INDUSTRIA FARMAC...,VÁLIDO,CEFTRIAXONA SÓDICA
30744,MEDICAMENTO,CEFUROXIMA SÓDICA,12/06/2023,GENÉRICO,174200015,01/07/2029,25351278156,CEFALOSPORINAS,44639493000180 - EUGIA PHARMA INDUSTRIA FARMAC...,VÁLIDO,CEFUROXIMA SÓDICA
30745,MEDICAMENTO,PROLUTEX,NaN,NOVO,123610094,01/09/2033,25351724777,PROGESTAGENOS SIMPLES,64711500000114 - UCB BIOPHARMA LTDA.,VÁLIDO,PROGESTERONA


In [4]:
# monologue
alphanumeric = lambda x: re.sub(r'\w*\d\w*', '', x)  # remover caractres especiais
punc = lambda x: re.sub(r'[%s]' % re.escape(string.punctuation), ' ', x)  # remover pontuações
remove_spacer = lambda x: x.replace('  ', ' ')  # remover espaço duplos

df['text_1'] = df['text'].map(alphanumeric).map(punc).map(remove_spacer)

In [ ]:
df.head()

In [ ]:
neg_tagged = df.query('sentiment == "negative"')['text_1']
pos_tagged = df.query('sentiment == "positive"')['text_1']

In [ ]:
lemmatizer = WordNetLemmatizer()
# instanciando a lista de stopwords
stopwords = set(nltk.corpus.stopwords.words('portuguese'))

def get_tag(tag):
    if tag.startswith("N"):
        pos = 'n'
    elif tag.startswith('V'):
        pos = 'v'
    else:
        pos = 'a'
    return pos

def to_tag(text):
    token, tag = tagger.tag(nltk.word_tokenize(text))[0]
    t = lemmatizer.lemmatize(token, get_tag(tag))
    
    if len(t) > 0 and t not in stopwords:
        return t.lower()

In [ ]:
pos_tagged

In [ ]:
list_tagged_pos = pos_tagged.map(lambda text: tagger.tag(nltk.word_tokenize(text)))

In [ ]:
list_tagged_neg = neg_tagged.map(lambda text: tagger.tag(nltk.word_tokenize(text)))

In [ ]:
pos_tagged[0:5], neg_tagged[0:5]

In [ ]:
# instanciando a lista de stopwords
stopwords = set(nltk.corpus.stopwords.words('portuguese'))

In [ ]:
def get_tag(tag):
    if tag.startswith("N"):
        pos = 'n'
    elif tag.startswith('V'):
        pos = 'v'
    else:
        pos = 'a'
    return pos
        
def remove_noise(list_tagged, stopwords = ()):
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = []

    for token, tag in list_tagged:
        token = lemmatizer.lemmatize(token, get_tag(tag))
        if len(token) > 0 and token not in stopwords:
            cleaned_tokens.append(token.lower())

    return cleaned_tokens

In [5]:
all_words = ' '.join([text for text in df['text_1']])

In [7]:
token_whitespace = nltk.tokenize.WhitespaceTokenizer()
token_frase = token_whitespace.tokenize(all_words) 
freq = nltk.FreqDist(token_frase)

In [8]:
freq

FreqDist({'de': 418258, 'que': 329920, 'e': 307033, 'o': 247224, 'um': 217802, 'a': 211431, 'é': 197296, 'filme': 152841, 'em': 133077, 'não': 131478, ...})